In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.filterwarnings("ignore")

from Parse_TFrecords import *
from define_model import *
from load_label import *
from utilities import *
import gc
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
from sklearn.metrics import classification_report, average_precision_score

# from libauc.losses import AUCMLoss 
# from libauc.optimizers import PESG 

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
BATCH_SIZE = 32

# pos = 72917
# neg = 263952

pos = 32162
neg = 149004

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
filename = ['copd_emory_train.tfrecords', 'copd_emory_val.tfrecords']
count = [0, 0]

raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

    count[example.features.feature['COPD'].int64_list.value[0]]+=1
    
count

In [ ]:
record_file_train = 'copd_emory_train.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(BATCH_SIZE)
.batch(BATCH_SIZE))

record_file_val = 'copd_emory_val.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.batch(BATCH_SIZE))

In [ ]:
archis = ['DenseNet121', 'InceptionV3', 'ResNet50V2', 'InceptionResNetV2', 'Xception', 'MobileNetV2', ]
for archi in archis:
    checkpoint_filepath = 'checkpoints_emory/checkpoint_BCE_{i}'.format(i=archi)
    monitor_ = 'val_loss'

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        save_weights_only=True,
        monitor=monitor_,
        mode='min')

    callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
                tf.keras.callbacks.EarlyStopping(mode='min', patience=3, monitor=monitor_),
                model_checkpoint_callback]

    model = load_model_from_pretrain(archi)

    loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    model.compile(loss=loss_func,
                     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics='AUC')
   
    model.fit(train_dataset, epochs=5, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

    del model
    gc.collect()

In [ ]:
BATCH_SIZE = 32
record_file_test = 'copd_emory_test.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_test = get_data_label('test')

In [ ]:
archis = ['DenseNet121', 'InceptionV3', 'ResNet50V2', 'InceptionResNetV2', 'Xception', 'MobileNetV2']

for archi in archis:
    checkpoint_filepath = 'checkpoints_emory/checkpoint_BCE_{i}'.format(i=archi)
    model = define_model(archi)
    model.load_weights(checkpoint_filepath)

    y_preds = model.predict(test_dataset)
    
    print(archi)
    
    test_CI(y_preds, y_test)
    
    print(average_precision_score(y_test, y_preds, average=None))
    
    thresh = get_thresh(y_test, y_preds, 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    gc.collect()